In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 31kB/s 
     |████████████████████████████████| 3.8MB 39.1MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 512kB 34.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=d35e8388a1e4825daa2b0d45d3270098e7de7943c73ea4571d0854025e694ed1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [4]:
import numpy as np
from numpy import argmax

In [5]:
import re
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>1:
        return u'<hem>'

    # if re.match(r"^[0-9]{2}0{3}$", word):
    #     return u'<postcode>'
    
    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    # word = word.replace("_"," ")
    
    return word

In [6]:
import numpy as np
import codecs
def read_data(input_file):
    with codecs.open(input_file, 'r', 'utf-8') as f:
        word_list = [] 
        words = []
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append( map_number_and_punct(line[0].lower()[0:31]) )
            else:
                word_list.append(words)
                words = []
    return word_list

In [7]:
#get char embedding
def read_char_vocab(file_path):
  char_vocab = []
  for line in open(file_path, encoding='utf-8'):
    char_vocab.append(line.splitlines()[0])      
  return char_vocab

def char_to_int(char_vocab_data):
  dic = {}
  index = -1
  for char in char_vocab_data:
    try:
      dic[char]
    except:
      index = index + 1
      dic[char]=index
  return dic
 
def get_char_encode(sentence,max_length_of_a_sentence,max_length_of_a_word):
  char_vocab_data = read_char_vocab("/content/drive/MyDrive/fast_text/VISCII_short.txt")
  dic = char_to_int(char_vocab_data)
  # sentence  words is a sentence | ['i','am','an']
  sentence_encoded = np.zeros([max_length_of_a_sentence,max_length_of_a_word]) # 25*25
  for j in range(len(sentence)):  
    word = sentence[j].lower()
    # integer_encoded = [char_to_int[char] for char in word]
    word_encoded = np.zeros(max_length_of_a_word)
    for k in range(len(word)):
      char = word[k]
      try:
        word_encoded[k]= dic[char]
      except:
        # print("error : " + str(char)+" "+str(word)+" "+str(len(word)))
        word_encoded[k]= dic['[unk]']
    # sentence encoded
    sentence_encoded[j] = word_encoded
  return sentence_encoded

def get_charEmbedd_form_encode(charEnocde):
  LEN_OF_VOCAB = 137
  shape = charEnocde.shape
  char_embedd = np.zeros([shape[0],shape[1],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      char_int = charEnocde[i,j]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,:] = onehot
  return char_embedd

# 
def get_char_embedd(test_word_sentences):
  char_embedd = []
  for sen_token in test_word_sentences:
    char_en = get_char_encode(sen_token,42,32)
    char_em = get_charEmbedd_form_encode(char_en)
    char_embedd.append(char_em)

  char_embedd = np.array(char_embedd)
  # print(char_embedd.shape)
  return char_embedd

In [8]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import math

In [9]:
# aphabet tag library
tags = {'pad': 0, 'LOCATION_HOMENUMBER': 1, 'LOCATION_STREET': 2, 'LOCATION_WARD': 3, 'LOCATION_DISTRICT': 4, 'LOCATION_PROVINCE': 5, 'LOCATION_COUNTRY': 6, 'LOCATION_POSTCODE': 7, 'LOCATION_NER': 8, 'OBJ': 9, 'OBJ_FEATURE': 10, 'PRE': 11, 'UNKNOW': 12 }
aphabet_tag = []
for key in tags:
  aphabet_tag.append(key)
# aphabet_tag = ['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW', 'LOCATION_SPECIAL']
print(aphabet_tag)

['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'LOCATION_NER', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW']


In [11]:
# load model=========================================================================
import os.path
from os import path

index = 0
link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)
while path.exists(link_model):
  print("model {}".format(index))
  index+=1
  link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)

index-=1
link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)
m = load_model(link_model)
#====================================================================================

model 0
model 1
model 2
model 3





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [12]:
import math

link_folder = "/content/drive/MyDrive/fast_text/split data/data_0"
index = 0
while path.exists(link_folder):
# while index==0:
  print("DATA {}".format(index))
  
  link_test = "/content/drive/MyDrive/fast_text/split data/data_{}/test/".format(index)
  
  test_word_sentences = read_data(link_test+'data_no_tag_pred.txt')
  val_char_embedd = get_char_embedd(test_word_sentences)
  # print(val_char_embedd.shape)

  val_tag = np.loadtxt(link_test+'tag_embedd.txt')
  val_tag = val_tag.reshape(int(val_tag.shape[0]/42), 42, val_tag.shape[1])
  # print(val_tag.shape)
  
  val_word = np.loadtxt(link_test+'word_embedd.txt')
  val_word = val_word.reshape(int(val_word.shape[0]/42), 42, val_word.shape[1])
  # print(val_word.shape)

  y_pred = m.predict([val_char_embedd, val_word])
  y_true = val_tag
  # print(y_pred.shape)
  # print(y_true.shape)

  y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
  y_pred = np.argmax(y_pred,axis=1)
  # print(y_pred.shape)

  y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
  y_true = np.argmax(y_true,axis=1)
  print(y_true.shape)

  # tìm acc
  a_true = 0
  a_false = 0
  for i in range(len(y_true)):
    if y_true[i]!=0 and y_true[i]!=12:
      if y_true[i]==y_pred[i]:
        a_true+=1
      else:
        a_false+=1
  
  acc = format(a_true/(a_true+a_false)*100, '.2f')
  print(' accuracy = {}'.format(acc))

  # lấy độ predict 
  dicts = []
  for element in y_true:
    if element not in dicts:
      dicts.append(element)
  dicts.sort()
  print(dicts)

  cnf_matrix = confusion_matrix(y_true, y_pred, labels=dicts)
  for i in range(len(cnf_matrix)):
    for j in range(len(cnf_matrix)):
      cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
  # print(cnf_matrix)

  # số tag đã đoán
  # print("số tag đã đoán của mỗi tag :")
  sum_doc = np.sum(cnf_matrix, axis = 0)
  num_pre = {}
  for i in range(len(cnf_matrix)):
    num_pre[aphabet_tag[i]] = sum_doc[i]
  # print(num_pre)

  # số tag đoán đúng
  # print("số tag đã đoán đúng của mỗi tag :")
  num_pre_true = {}
  for i in range(len(cnf_matrix)):
    num_pre_true[aphabet_tag[i]] = cnf_matrix[i][i]
  # print(num_pre_true)

  # số đối tượng có sẵn của mỗi tag
  # print("số tag có sẵn của mỗi tag : ")
  sum_ngang = np.sum(cnf_matrix, axis = 1)
  num_tag = {}
  for i in range(len(cnf_matrix)):
    num_tag[aphabet_tag[i]] = sum_ngang[i]
  # print(num_tag)
  
  # convert
  num_pre_cv = [ num_pre[tag] for tag in num_pre]
  num_pre_true_cv = [ num_pre_true[tag] for tag in num_pre_true]
  num_tag_cv = [ num_tag[tag] for tag in num_tag]

  X = np.empty([1, 3, 13])
  X[0, 0, :] = num_pre_true_cv
  X[0, 1, :] = num_pre_cv
  X[0, 2, :] = num_tag_cv
  fileout = open("/content/drive/MyDrive/model/evaluate/evaluate_test_data.txt", 'a+', encoding='utf8')
  np.savetxt(fileout, X[0])
  fileout.close()
  
  index+=1
  link_folder = "/content/drive/MyDrive/fast_text/split data/data_{}".format(index)

DATA 0
(37800,)
 accuracy = 98.89
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 1
(37800,)
 accuracy = 98.73
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 2
(37800,)
 accuracy = 98.73
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 3
(31626,)
 accuracy = 98.34
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
